# Photogrammetry API

<div class="alert alert-block alert-warning">

<b>The Photogrammetry API is highly experimental and some features may not be available or function correctly at the moment. Please use this API with care, as we issue no guarantees this API won't break.
</b> 
</div>

In this tutorial we will go through how to use Cognite's Photogrammetry API to create contextualised 3D models using images.

The API reference is available here: [https://doc.cognitedata.com/api/playground/](https://doc.cognitedata.com/api/playground/)


### Background

In order to create contextualised 3D models we have developed a pipeline with several modular parts. First, images are uploaded as a zip file by the user. These images go through a quality control step, where low quality images (blurry) are filtered out. Next, the 3D reconstruction and tag detection steps are started. The former is done by using a method called photogrammetry, which uses overlapping images from a real-world object or scene to create a 3D model.

The tag detection is done in two steps. First we use Convolution Neural Networks (CNN) for detection of tags, and thereafter optical character recognition (OCR) is used for recognizing the tag text. In the last step, the image pixel coordinates extracted by the tag detector are mapped to 3D locations.


<img src="images/pipeline.png" width="600" align="center">


3D data genereated by photogrammetry is organized in models and revisions in Cognite Data Fusion (CDF), similar to how other 3D data is stored in CDF:

   * Models are placeholders for a set of revisions.

   * Revisions contain the 3D data.

Revisions can for instance represent time evolution of the 3D model, or different versions of the model (high detail vs low detail).

When you create a new revision and upload images, the reconstruction will start immediately. Depending on the the quality and number of images, this can take some time. A revision can have status `CREATED`, `IN_PROGRESS`, `SUCCESS` or `FAILED`, which can be tracked during processing.

## Step-by-step example

The photogrammetry API is currently only enabled for a few selected tenants. If you want access from other tenants, please contact Cognite to get it enabled or open a pull request in this repo.

First, we make the imports we'll need for this tutorial and use an API key to authenticate. Make sure that you have set the API key and client name as environment variables and have installed the latest version of [Cognite Python SDK](https://cognite-docs.readthedocs-hosted.com/projects/cognite-sdk-python/en/latest/).

In [ ]:
import os
from urllib.request import urlretrieve
from cognite.client import CogniteClient

BASEURL = "/api/playground"
os.environ["COGNITE_DISABLE_GZIP"] = "1"
client = CogniteClient(api_key=os.environ['COGNITE_API_KEY'], 
                       client_name=os.environ['COGNITE_CLIENT_NAME'])

### List Photogrammetry models

List all available Photogrammetry models:

In [ ]:
def list_photogrammetry_models():
    response = client.get(url=f"{BASEURL}/photogrammetry")
    return response.json()

list_photogrammetry_models()

### Create and retrieve a Photogrammetry model

To create a Photogrammetry model you need to provide the `name` field. 

In [ ]:
name = "my first model" # Your model name

In [ ]:
def create_photogrammetry_model(name):
    response = client.post(url=f"{BASEURL}/photogrammetry",
                           json={"items": [{"name": name}]})
    model = response.json()["items"][0]
    return model

model = create_photogrammetry_model(name)
model

Note that, multiple models can be created in one request but here we only create one single model. Once created, you can retrieve the model by id:

In [ ]:
def get_photogrammetry_model(p_id):
    response = client.get(url=f"{BASEURL}/photogrammetry/{p_id}")
    return response.json()

get_photogrammetry_model(p_id = model["id"])

### List Photogrammetry model revisions

The list of all available revisions in the newly created Photogrammetry model should be empty:

In [ ]:
def list_revisions(p_id):
    response = client.get(url=f"{BASEURL}/photogrammetry/{p_id}/revisions")
    return response.json()

list_revisions(model["id"])

### Create and retrieve a Photogrammetry model revision

When creating a revision, you can choose to detect tags in the images that are used to generate the 3D model. This can be done by setting `detect_tags=True` (default is `False`):

In [ ]:
detect_tags = True

In [ ]:
def create_revision(p_id, detect_tags):
    response = client.post(url=f"{BASEURL}/photogrammetry/{p_id}/revisions", 
                           json={"items": [{"detectTags": detect_tags}]})
    revision = response.json()["items"][0]
    return revision

revision = create_revision(p_id=model["id"], detect_tags=detect_tags)

As for Photogrammetry models, multiple revisions can be created in one request, but here we create one signle revision (you need to create a new revision every time you upload a new set of images). Once created, you can retrieve the revision by id:

In [ ]:
def get_revision(p_id, r_id):
    response = client.get(url=f"{BASEURL}/photogrammetry/{p_id}/revisions/{r_id}")
    return response.json()

get_revision(p_id=model["id"], r_id=revision["id"])

This revision will now be part of the model we created earlier:

In [ ]:
list_revisions(model["id"])

### Upload images

To upload images, you need to provide the upload url for the revision and the filepath for the zip file containing the images. The reconstruction will start immediately after the upload. We have created a test dataset, which you can download from [here](https://drive.google.com/open?id=1VaIURFCuv0BT2ny_-wcz6Kucayj5qkIk) or you can use your own images (*remember to change the zip file path!*). 


<div class="alert alert-block alert-info"><b>
    
* The supported image formats are: jpeg, jpg, jpe, and png.
    
    
* The maximum number of images you can have in a model is 300.


* Maximum size of a single image: 128 MB 
</b></div>


In [ ]:
def upload_images(upload_url, zip_file_path):
    import requests
    headers = {"content-length": str(os.path.getsize(zip_file_path))}
    
    with open(zip_file_path, "rb") as file:
        requests.put(upload_url, data=file, headers=headers)
        
zip_file_path = "/path/to/zipfile.zip" # CHANGE ME!

upload_images(revision["uploadURL"], zip_file_path)

### Retrieve the 3D file

To check whether your 3D model is ready, use `get_revision()` to see if the reconstruction job is finished. If that is the case, `status` will be `SUCCESS` and you get a download URL link that can be used to download the 3D file (fbx file).


<div class="alert alert-block alert-info"><b>
It takes around 15-20 minutes to process and create a 3D model using the dataset provided above.
</b></div>

In [ ]:
response = get_revision(p_id=model["id"], r_id=revision["id"])

output_filename = "output.fbx"

if response["status"] == "SUCCESS":
    download_url = response["downloadURL"]
    urlretrieve(download_url, output_filename)
    
response

### Retrieve tag locations

All detected labels can be retrieved by:

In [ ]:
def get_tags(p_id, r_id):
    response = client.get(url=f"{BASEURL}/photogrammetry/{p_id}/revisions/{r_id}/tags")
    return response.json()

get_tags(p_id=model["id"], r_id=revision["id"])

### Delete the photogrammetry model

To avoid crowding it might be a good idea to remove old/unused photogrammetry models and revisions. A revision can be deleted by:

In [ ]:
def delete_revision(p_id, r_id):
    response = client.post(url=f"{BASEURL}/photogrammetry/{p_id}/revisions/delete",
                           json={"items": [{"id": r_id}]})
    return response.json()

delete_revision(p_id = model["id"], r_id=revision["id"])

A photogrammetry model can be deleted by (will also delete all its revisions): 

In [ ]:
def delete_photogrammetry(p_id):
    response = client.post(url=f"{BASEURL}/photogrammetry/delete",
                           json={"items": [{"id": p_id}]})
    return response.json()

delete_photogrammetry(p_id = model["id"])

## Useful links


* [Autodesk ReMake - How to Take Photos for Photogrammetry](https://www.youtube.com/watch?v=D7Torjkfec4)

* [The Art of Photogrammetry: How To Take Your Photos](https://www.tested.com/art/makers/460142-art-photogrammetry-how-take-your-photos/)



